In [ ]:
## Notebook per il training ##

# Importazione delle librerie necessarie

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from io import StringIO

# Importazione degli url attraverso la libreria Request

red_wine_url= 'https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/wine_quality/winequality-red.csv'
white_wine_url = 'https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/wine_quality/winequality-white.csv'

# Definizione delle GET attraverso la Requests per avere i dati
response_red = requests.get(red_wine_url)
response_white = requests.get(white_wine_url)

# Check per vedere se le GET hanno dato esito favorevole o meno, restituendomi i dati richiesti
if response_red.status_code == 200 and response_white.status_code == 200:
    df_red_wine = pd.read_csv(StringIO(response_red.text), sep=';')
    df_white_wine = pd.read_csv(StringIO(response_white.text), sep=';')

    display(df_red_wine)
    display(df_white_wine)
# Restituisce gli status code per ciascun URL se le richieste non hanno ritrovato i dati    
else:
    print('Errore. Status code (Primo URL):', response_red.status_code)
    print('Errore. Status code (Secondo URL):', response_white.status_code)

In [ ]:
# Prima di iniziare a lavorare sui dati, è mia prassi fare un backup dei df originali
# Copia di backup dei df originali

df_r_wine = df_red_wine.copy()
df_w_wine = df_white_wine.copy()

In [ ]:
# Considerazioni preliminari:
# i due df hanno una ingente disparità di record, rendendo così difficoltosa un'analisi approfondita basata sulla tipologia (vino rosso e bianco)
# si tratta di tipologie diverse dello stesso vino "Vinho Verde" (fonte: punto 4, https://github.com/FabioGagliardiIts/datasets/blob/main/wine_quality/winequality.names)
# lo scopo è quello di predirre la qualità del vino (a prescindere, quindi, dal fatto che si tratti di una variante di rosso o di bianco)
# Fatte queste premesse, mi è sembrato opportuno unire i due df in un unico dataset per avere una migliore visione d'insieme e per porre in essere una predizione più concreta
df_r_wine['type'] = 0
df_w_wine['type'] = 1

display(df_r_wine)
display(df_w_wine)

frames = [df_r_wine, df_w_wine]
df_wine = pd.concat(frames)
display(df_wine)

<h1> Legenda delle colonne </h1>

1 Fixed acidity. The predominant fixed acids in wine, such as tartaric, succinic, citric, and malic acids. <br>
2 Volatile acidity. The high acetic acid present in wine, which causes an unpleasant vinegar taste. <br>
3 Citric acid. A weak organic acid used to increase the freshness and flavor of wine.<br>
4 Residual sugar. The amount of sugar left after fermentation. <br>
5 Chlorides. The amount of salt in wine. The lower chloride rate creates better quality wines. <br>
6 Free sulfur dioxide. SO2 is used for preventing wine from oxidation and microbial spoilage. <br>
7 Total sulfur dioxide. The amount of free and bound forms of SO2.<br>
8 Density. Depends on the alcohol and sugar content. Better wines usually have lower densities. <br>
9 pH. Used to check the level of acidity or alkalinity of wine. <br>
10 Sulfates. An antibacterial and antioxidant agent added to wine. <br>
11 Alcohol. The percentage of alcohol in wine. A higher concentration leads to better quality.<br>
12 Type. 0 Red wine, 1 White wine.

<h1> Analisi EDA </h1>
- Controllo per eventuali valori nulli <br>
- Controllo degli outliers <br>
- Gestione degli outliers <br>
- Data visualization per capire l'andamento dei dati <br>
- Matrice di correlazione per inidividuare quali potrebbero essere le features da mantenere per il traning del modello <br>


In [ ]:
# Check per vedere se ci sono valori nulli
# Utilizzo una funzione che mostra quanti valori nulli ci sono - o potrebbero esserci- all'interno del df

def display_null_counts(df):
    null_counts = df.isna().sum()
    display(null_counts)
    print('\n')

display_null_counts(df_wine)

In [ ]:
# Utilizzo il metodo .describe() per iniziare ad analizzare il contenuto del df

df_wine.describe()

<h1> Outliers </h1>
<h3> Da una preliminare analisi dei dati, è possibile notare come ci sia/siano record nella colonna 'quality' (colonna target) con valori molto alti e molto bassi. Ho quindi optato per capire quali e quanti siano quei record </h3>

In [ ]:
# Definisco un treshold nella colonna quality oltre il quale i record sono classificati come outliers
# Ho optato per definire i record oltre il valore 7 come outliers
outliers = df_wine[df_wine['quality'] >= 7]

# Grafico per capirne la distribuzione
plt.hist(outliers['quality'], bins=5, edgecolor='black', color='green')
plt.xlabel('Quality')
plt.ylabel('Count')
plt.title('Distribuzione dei Record con Quality >= 7')
plt.show()

# Separo gli outliers: voglio sapere quanti record ci sono per ogni valore >= 7
# Numero dei record con valore 7
count_quality_7 = (df_wine['quality'] == 7).sum()
records_quality_7 = df_wine[df_wine['quality'] == 7]

# Numero record con valore 8
count_quality_8 = (df_wine['quality'] == 8).sum()
records_quality_8 = df_wine[df_wine['quality'] == 8]

# Numero record con valore 9
count_quality_9 = (df_wine['quality'] == 9).sum()
records_quality_9 = df_wine[df_wine['quality'] == 9]

print(f"Numero di record con quality = 7: {count_quality_7}")
print(f"Numero di record con quality = 8: {count_quality_8}")
print(f"Numero di record con quality = 9: {count_quality_9}")

In [ ]:
# preliminare dataviz della distribuzione dei dati 

# N.B. 
# ho commentato la successiva linea di codice solo per risparmiare tempo e potere computazionale

#sns.pairplot(df_wine)

In [ ]:
# Matrice di correlazione per avere una rappresentazione grafica di quale sia (o siano) la feature con una correlazione più forte con la colonna target (quality)
# ho utilizzato una variabile 'ax' per aumentare le dimensioni della matrice di correlazione

fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(df_wine.corr(), annot=True, ax = ax)

In [ ]:
# dataviz a istogrammi per vedere la distribuzione dei dati del df
df_wine.hist(bins=25, figsize = (10,10))
plt.show()

In [ ]:
# Definita la matrice di correlazione, prendo in considerazione la colonna target (quality) con la feature con maggiore correlazione (alcohol)
# dataviz per mostrare a schermo il grafico che prende in considerazione la colonna target con la main feature del df
plt.figure(figsize = (10,6))
plt.bar(df_wine['quality'], df_wine['alcohol'], color = 'green')
plt.xlabel('quality')
plt.ylabel('alcohol')

In [ ]:
# determino con un valore arbitrario (in questo caso 0,7) quale sia/siano le altre features che hanno una correlazione forte 
plt.figure(figsize=(12, 12))
sns.heatmap(df_wine.corr() > 0.7, annot=True, cbar=False, cmap='tab20c')
plt.show()

In [ ]:
# Constatato il fatto che ci sono features che hanno una forte correlazione tra loro opto per:
# rimuovere la colonna 'total sulfur dioxide' dal df
df_wine = df_wine.drop('total sulfur dioxide', axis = 1)

display(df_wine)

<h1> Considerazioni sulla selezione delle features per il modello: </h1>
- l'obiettivo è quello di predirre se un vino è di buona o di scarsa qualità, ergo sarebbe opportuno semplificare la classificazione creando una nuova colonna 'best quality'<br>
- l'idea è quella di settare un treshold sulla colonna quality per distinguere se un vino è di ottima o di scarsa qualità <br>
- la colonna 'best quality' conterrebbe valori 0 o 1 in base a quanto detto nel punto sopra <br>
- nell'analisi EDA è emersa una considerevole disparità tra la qualità media dei vini rispetto a quelli di alta o bassa qualità; in questo caso creare una colonna 'best quality' bilancerebbe la distribuzione dei dati per porre in essere una modellazione più accurata <br>
- gestione degli outliers: creando la colonna 'best quality' si eviterebbe di eliminare o riqualificare gli outliers, mantenendo così i valori originali <br>

In [ ]:
# Creazione di una nuova colonna ('best quality') che iterando sulla colonna quality assegna il valore 1 (se la qualità del vino è > 5) o il valore 0 (se inferiore di 5)
df_wine['best quality'] = [1 if x > 5.8 else 0 for x in df_wine.quality]
display(df_wine)

In [ ]:
# Dataviz per la distribuzione dei dati tra la colonna target e quella delle features
# Aggiunta un treshold per visualizzare i valori 'quality' dai valori 'best quality'
# la treshold è settata a 5.8 che è la media dei valori
df_wine.plot.scatter(x="quality", y="best quality")
plt.axvline(x=5.8, color='r', linestyle='--', label='Threshold: 5.8')
plt.show()

<h1> Implementazione del modello </h1>

**Considerata la distribuzione dei dati e l'obiettivo finale, ritengo opportuno utilizzare la Logistic Regression per la classificazione dei vini**
<h6> Colonna target: best quality </h6>
<h6> Colonna features: quality</h6>

In [ ]:
# Importazione delle librerie necessarie per l'implementazione del modello di ml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, r2_score

<h2> Split di Training e Testing </h2> 

In [ ]:
# Selezione delle features e della colonna target
features = df_wine.drop(['quality', 'best quality'], axis=1)
target = df_wine['best quality']

# split di training e testing delle features e del target
x_train, x_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=40)

# shape delle tuple di training e di testing
x_train.shape, x_test.shape

<h2> Normalizzazione </h2>

In [ ]:
# Normalizzazione dei valori (rispettivamente di training e di testing) utilizzando il MinMaxScaler di sklearn 
norm = MinMaxScaler()
xtrain = norm.fit_transform(x_train)
xtest = norm.transform(x_test)
#print(xtrain)
#print(xtest)

In [ ]:
# inizializzazione del modello di regressione logistica
logistic_model = LogisticRegression()

# training del modello
logistic_model.fit(xtrain, y_train)

# previsione sui test
y_pred_logistic = logistic_model.predict(xtest)

# definizione della precisione del modello utilizzando la libreria accuracy_score di sklearn
accuracy = accuracy_score(y_test, y_pred_logistic)
print('Precisione del modello:', accuracy)

print('\n')

# inizializzazione della matrice di confusione per la visualizzazione dei risultati
conf_matrix_logistic = confusion_matrix(y_test, y_pred_logistic)

# Definizione dell'errore quadrato medio
mse = mean_squared_error(y_test, y_pred_logistic)
print('MSE:', mse)

# definizione di r2
r2 = r2_score(y_test, y_pred_logistic)
print('R2:', r2)

In [ ]:
# Dataviz: grafico della matrice di confusione

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_logistic, annot=True, fmt="d", cmap="OrRd_r", cbar=False)

# Precisazione dei label per la matrice di confusione
plt.text(0.3, 0.3, 'TP (True Positive)', fontsize=12, color='black', horizontalalignment='center', verticalalignment='center')
plt.text(1.3, 0.3, 'FP (False Positive)', fontsize=12, color='black', horizontalalignment='center', verticalalignment='center')
plt.text(0.3, 1.3, 'FN (False Negative)', fontsize=12, color='black', horizontalalignment='center', verticalalignment='center')
plt.text(1.3, 1.3, 'TN (True Negative)', fontsize=12, color='black', horizontalalignment='center', verticalalignment='center')

plt.xlabel('Valori Predetti')
plt.ylabel('Valori Reali')
plt.title('Matrice di Confusione (Logistic Regression)')
plt.show()

<h1> Ridefinizione dei titoli delle colonne </h1>

In [ ]:
# Una volta addestrato il modello, ho rinominato i titoli delle colonne del df, sostituendo gli spazi con '_'
# Questa modifica mi consente di utilizzare i nomi delle colonne come features nel BaseModel quando costruirò l'API per esporre il modello
df_wine.columns = df_wine.columns.str.replace(' ', '_')

In [ ]:
# Mostra il df con le modifiche apportate ai nomi delle colonne
display(df_wine)

In [ ]:


# Supponiamo che df_wine sia il tuo DataFrame
# Specifica il percorso completo del file CSV, incluso il nome del file
percorso_file_csv = '/Users/albertopuggioni/PycharmProjects/ai_ml_esame_finale/CSV/wine_data.csv'

# Utilizza il metodo to_csv() per esportare il DataFrame nella cartella specificata
df_wine.to_csv(percorso_file_csv, index=False)  # L'argomento index=False impedisce di scrivere l'indice nel CSV

<h1> Salvataggio del modello </h1>

<h1> Ho provato ad utilizzare la libreria di sklearnpmml2 per esportare e salvare il modello, ma mi dava un errore di compatibilità con la versione del converter. Ho inoltre provato a fare il downgrade di sklearn alla versione 1.2.2, ma senza successo.
Ho quindi scelto di utilizzare la libreria joblib per salvare ed esportare il modello </h1>

In [ ]:
# Importo la libreria per salvare il modello
#from sklearn2pmml import sklearn2pmml

# Nomino il modello e lo salvo nella directory 'modello' del progetto
#pmml_modello_wine_quality = '/Users/albertopuggioni/PycharmProjects/ai_ml_esame2/modello/logistic_regression_model.pmml'

# conversione del modello in pmml 
#sklearn2pmml(logistic_model, pmml_modello_wine_quality)

# messaggio che conferma l'versione e salvataggio
#print(f"Successo: Modello convertito e salvato come '{pmml_modello_wine_quality}'")

In [ ]:
# importazione della libreria joblib per salvare il modello addestrato
import joblib

# valorizzo il modello e lo inserisco nella cartella appropriata del progetto
model_wine_quality = "/Users/albertopuggioni/PycharmProjects/ai_ml_esame_finale/modello/modello_logistic_regression.pkl"

# salvataggio del modello + messaggio di conferma dell'avvenuto salvataggio
joblib.dump(logistic_model, model_wine_quality)
print(f"Modello salvato con successo in: {model_wine_quality}")